# Retrieval Augmented Generation (RAG) using Azure AI SDK

## Objective

This notebook demonstrates the following:

- Create an index on Azure Cognitive search or FAISS from your local files
- Use the index in a chatbot built using Azure OpenAI and Langchain

This tutorial uses the following Azure AI services:

- Access to Azure OpenAI Service - you can apply for access [here](https://go.microsoft.com/fwlink/?linkid=2222006)
- Azure Cognitive Search service - you can create it from instructions [here](https://learn.microsoft.com/azure/search/search-create-service-portal)
- An Azure AI Studio project - go to [aka.ms/azureaistudio](https://aka.ms/azureaistudio) to create a project
- A connection to the Azure Cognitive Service in your project

## Time

You should expect to spend 10-20 minutes running this sample. 

## About this example

This sample shows how to create an index and consume it to answer questions based on your data (aka RAG pattern). It demonstrates how to create an index from local files and folders, how to store that index in Azure Cognitive Search or in [FAISS](https://ai.meta.com/tools/faiss). The index gets created locally and can then be registered to your AI Studio project. Once registered, it can be retrieved and consumed to answer questions. The sample shows how to build a simple QnA script to answer questions.

This sample is useful for developers and data scientists who wish to use their data with LLMs to build QnA bots, co-pilots. Basically anyone interested in using the RAG pattern.

### Data

This sample uses files from the folder `notebooks/rag/rag-e2e/data/product-info` in this repo. You can clone this repo or copy this folder to make sure you have access to these files when running the sample.

## Before you begin

### Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install the packages
%pip install azure-identity azure-ai-generative[faiss] azure-search-documents azure-ai-resources
%pip install langchain

### Parameters

Lets initialize some variables. For `subscription_id`, `resource_group_name` and `project_name`, you can go to the Project Overview page in the AI Studio. Replace the items in <> with values for your project.

In [ ]:
# project details
subscription_id: str = "<your-subscription-id>"
resource_group_name: str = "<your-resource-group>"
project_name: str = "<your-project-name>"

# Azure Cognitive Search Connection
acs_connection_name: str = "<your-acs-connection>"

# models used for embedding and deployment
embedding_model_deployment: str = "text-embedding-ada-002"
chat_model_deployment: str = "gpt-35-turbo"

should_cleanup: bool = False

## Connect to your project

To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace. To get the required details, you can go to the Project Overview page in the AI Studio. 

In [ ]:
import json
from pathlib import Path

config = {
    "subscription_id": subscription_id,
    "resource_group": resource_group_name,
    "project_name": project_name,
}

p = Path("config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

Let us connect to the project

In [ ]:
from azure.ai.resources.client import AIClient
from azure.identity import DefaultAzureCredential

# connects to project defined in the first config.json found in this or parent folders
client = AIClient.from_config(DefaultAzureCredential())

## Retrieve Azure OpenAI and Cognitive Services Connections
We will use an Azure Open AI service to access the LLM and embedding model. We will also use an Azure Cognitive Search to store the index. Let us get the details of these from your project.

In [ ]:
# Get the default Azure Open AI connection for your project
default_aoai_connection = client.get_default_aoai_connection()
default_aoai_connection.set_current_environment()

# Get the Azure Cognitive Search connection by name
default_acs_connection = client.connections.get(acs_connection_name)
default_acs_connection.set_current_environment()

## Build an Index locally
We will use files from the data directory and build an index. The index will be created on the machine where this sample is run.

You can index files of type `.md, .txt, .html, .htm, .py, .doc, .docx, .ppt, .pptx, .pdf, .xls, .xlsx`. All other file types will be ignored.

### Build an Azure Cognitive Search Index
Let us create an Azure Cognitive Search Index. The below step will chunk and embed your documents locally and then add it to an index in the Azure Cognitive Search Service.

In [ ]:
from azure.ai.resources.operations import LocalSource, ACSOutputConfig
from azure.ai.generative.index import build_index

index_name = "product-info-acs-index"
# build the index
acs_index = build_index(
    output_index_name=index_name,  # name of your index
    vector_store="azure_cognitive_search",  # the type of vector store - in our case it is ACS
    # we are using ada 002 for embedding
    embeddings_model=f"azure_open_ai://deployment/{embedding_model_deployment}/model/text-embedding-ada-002",
    index_input_config=LocalSource(input_data="data/product-info/"),  # the location of your file/folders
    acs_config=ACSOutputConfig(
        acs_index_name=index_name + "-store",  # the name of the index store inside the azure cognitive search service
    ),
)

### Build a FAISS Index
The same index can also be created on FAISS. The below step will chunk and embed your documents locally and then add it a FAISS index which is a file based index stored on your local machine.

In [ ]:
from azure.ai.resources.operations import LocalSource
from azure.ai.generative.index import build_index

faiss_index_name = "product-info-faiss-index"
# build the index
faiss_index = build_index(
    output_index_name=faiss_index_name,  # name of your index
    vector_store="faiss",  # the type of vector store - in our case it is ACS
    # we are using ada 002 for embedding
    embeddings_model=f"azure_open_ai://deployment/{embedding_model_deployment}/model/text-embedding-ada-002",
    index_input_config=LocalSource(input_data="data/product-info/"),  # the location of your file/folders
)

## Register the index
Register the index so that it shows up in the AI Studio Project. 

In the below step we will register the Azure Cognitive Search Index which we created, but the same method can be used to register a FAISS index as well.

In [ ]:
client.indexes.create_or_update(acs_index)

## Consuming the index
To consume an Index, you need to retrieve it from the project. You can then use langchain to query the index. In this sample, we will use the Azure Cognitive Search Index which we created, but the same can be applied to any index in your project.

### Retrieve the index from your project as a langchain retriever
Let us get the index from your project as a langchain retriever. We will use the langchain retriever to build a chat function

In [ ]:
from azure.ai.generative.index import get_langchain_retriever_from_index

# retrieve ML Index from your project
index_name = "product-info-acs-index"
# this function needs a path. Convert index name to path by adding -mlindex at the end
index_langchain_retriever = get_langchain_retriever_from_index(index_name + "-mlindex")

### Build a QnA function
We will build a QnA function which accepts a user question and provides an answer from the index data.
This function uses the LLM as defined in the `AzureChatOpenAI` constructor. It uses the index as a langchain_retriever to query the data. We build a prompt Template which accepts a context and a question. We use langchain's `RetrievalQA.from_chain_type` to put all these together and get us the answers.

In [ ]:
def qna(question: str, temperature: float = 0.0, prompt_template: object = None) -> str:
    from langchain import PromptTemplate
    from langchain.chains import RetrievalQA
    from langchain.chat_models import AzureChatOpenAI

    llm = AzureChatOpenAI(
        deployment_name=chat_model_deployment,
        model_name=chat_model_deployment,
        temperature=temperature,
    )

    template = """
    System:
    You are an AI assistant helping users with queries related to outdoor outdoor/camping gear and clothing.
    Use the following pieces of context to answer the questions about outdoor/camping gear and clothing as completely, 
    correctly, and concisely as possible.
    If the question is not related to outdoor/camping gear and clothing, just say Sorry, I only can answer question 
    related to outdoor/camping gear and clothing. So how can I help? Don't try to make up an answer.
    If the question is related to outdoor/camping gear and clothing but vague ask for clarifying questions.
    Do not add documentation reference in the response.

    {context}

    ---

    Question: {question}

    Answer:"
    """
    prompt_template = PromptTemplate(template=template, input_variables=["context", "question"])

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=index_langchain_retriever,
        return_source_documents=True,
        chain_type_kwargs={
            "prompt": prompt_template,
        },
    )

    response = qa(question)

    return {
        "question": response["query"],
        "answer": response["result"],
        "context": "\n\n".join([doc.page_content for doc in response["source_documents"]]),
    }

Let us ask a question to make sure we get an answer.

In [ ]:
result = qna("Which tent has the highest rainfly waterproof rating?")
print(result["answer"])

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    # add cleanup steps if needed
    pass